In [1]:
import respy as rp
import pandas as pd
import numpy as np

In [8]:
params, options = rp.get_example_model("kw_94_one", with_data=False)
options

{'estimation_draws': 200,
 'estimation_seed': 500,
 'estimation_tau': 500,
 'interpolation_points': -1,
 'n_periods': 40,
 'simulation_agents': 1000,
 'simulation_seed': 132,
 'solution_draws': 500,
 'solution_seed': 1,
 'core_state_space_filters': ["period > 0 and exp_{i} == period and lagged_choice_1 != '{i}'",
  "period > 0 and exp_a + exp_b + exp_edu == period and lagged_choice_1 == '{j}'",
  "period > 0 and lagged_choice_1 == 'edu' and exp_edu == 0",
  "lagged_choice_1 == '{k}' and exp_{k} == 0",
  "period == 0 and lagged_choice_1 == '{k}'"],
 'covariates': {'constant': '1',
  'exp_a_square': 'exp_a ** 2',
  'exp_b_square': 'exp_b ** 2',
  'at_least_twelve_exp_edu': 'exp_edu >= 12',
  'not_edu_last_period': "lagged_choice_1 != 'edu'",
  'edu_ten': 'exp_edu == 10'}}

In [3]:
simulate = rp.get_simulate_func(params, options)

In [29]:
params.loc[("eta", "eta"), :] = 0.01
#print(params.get("delta", "value"))
params.get(("eta", "eta"), 99)


99

In [ ]:
for eta in np.linspace(0, 0.01, 11):
    params.loc[("eta", "eta"), :] = eta
    df = simulate(params)
    stat = df.groupby("Identifier")["Experience_Edu"].max().mean()
    print(eta, stat)

array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])